<a href="https://www.kaggle.com/code/patrickleal/men-and-women-clothing-data-analysis?scriptVersionId=143242125" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
!pip install chardet

## **Importing libraries**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# libraries to help read the files
import os
from chardet import detect
import glob

# Libraries for data visualization.
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# **Creating Functions to read the files**

**Function to combine all DataFrames**

In [ ]:

def concatenate_dataframes(files_path):
    ''' This Function combine all DataFrames into one.'''
    
    # Initialize an empty list to hold DataFrames
    dfs = []

    for file in files_path:
        # Detect the encoding of the file
        with open(file, 'rb') as rawdata:
            detection_result = detect(rawdata.read(10000))

        # save the character encoding of the CSV files
        encoding = detection_result['encoding']

        # Read the CSV file using the detected encoding
        df = pd.read_csv(file, encoding=encoding)

        # Create a new column for each dataframe
        create_product_category(file, df)

        # realocate the new column position
        df = df[['product_category'] + [col for col in df.columns if col != 'product_category']]

        # Append the dataframe to the list
        dfs.append(df)

     #  Concatenate all DataFrames into one
    raw_combined_df = pd.concat(dfs, ignore_index=True)
    return raw_combined_df

**Function to create a column**

In [ ]:
def create_product_category(file_path, dataframe):
    '''This function create a new column for each original dataset based on the product category.'''
    
    df = dataframe
    
    name_with_extension = os.path.basename(file_path) # read the complete file path string
    name_without_extension = os.path.splitext(name_with_extension)[0] # save only the file name without extension

    df['product_category'] = name_without_extension
    
    return df

***

# **Reading the Files**

In [ ]:
# create a list with all the men files path
men_files_path = sorted(glob.glob('/kaggle/input/zara-dataset-men-and-women-clothing/Men/Men/*.csv'))

# create a list with all the women files path
women_files_path = sorted(glob.glob('/kaggle/input/zara-dataset-men-and-women-clothing/Women/Women/*.csv'))

In [ ]:
# Creates a dataframe with all men files combined
men_combined_df_raw = concatenate_dataframes(men_files_path)

# Creates a dataframe with all women files combined
women_combined_df_raw = concatenate_dataframes(women_files_path)

**Checking the men DataFrame**

In [ ]:
men_combined_df_raw.head(10)

In [ ]:
men_combined_df_raw.columns

In [ ]:
men_combined_df_raw.info()

In [ ]:
men_combined_df_raw.describe(include='all')

**Checking the women DataFrame**

In [ ]:
women_combined_df_raw.head(10)

In [ ]:
women_combined_df_raw.columns

In [ ]:
women_combined_df_raw.info()

In [ ]:
women_combined_df_raw.describe(include='all')

Looks like we can drop the same columns from both datasets:
- 'Unnamed: 0	'
- 'Link'
- 'Product_Image'

***

# **1) Data Cleanig**

## **1.1) Men dataset**

In [ ]:
# creating a copy of the raw dataset and then removing any unwanted columns

men_df = men_combined_df_raw.copy()
men_df = men_df.drop(columns=['Unnamed: 0', 'link', ' product_images'])
men_df

In [ ]:
men_df.info()

**TO DO:**

1. Check if there are any rows with missing values in the 'price' column.
1. Check how many rows have missing values in the 'price' column per category.
1. Decide whether it's worthwhile to remove the NA values or replace them.

In [ ]:
# checking missing data in the price column 

men_df[men_df['price'].isna()]

In [ ]:
# checking only the BEAUTY category
men_df.loc[men_df['product_category'] == 'BEAUTY']

In [ ]:
# checking only the SUITS category
men_df.loc[men_df['product_category'] == 'SUITS']

The entire row in the BEAUTY category is missing, so I will remove the entire row. And the entire SUITS category are missing, so i will remove the entire category.

In [ ]:
men_df =  men_df.dropna(subset=['price', 'product_name'])
men_df.head()

In [ ]:
# renaming the column price
men_df = men_df.rename(columns={'price' : 'original_price(INR)'})

# removing characters
men_df['original_price(INR)'] = men_df['original_price(INR)'].str.replace('₹ ', '')\
                                                                 .str.replace(',', '')
men_df['original_price(INR)'] = men_df['original_price(INR)'].str.strip()

# converting to float
men_df['original_price(INR)'] = pd.to_numeric(men_df['original_price(INR)'], errors='coerce')

In [ ]:
men_df.head()

In [ ]:
# checking missing values
men_df[men_df['original_price(INR)'].isna()]

There is no missing values in the column 'original_price(INR)'

Now I will create a new price column with values in USD.

Average exchange rate in 2023: 0.0122 USD

source: https://www.exchangerates.org.uk/INR-USD-spot-exchange-rates-history-2023.html

In [ ]:
men_df['price(USD)'] = men_df.iloc[:, 2] * 0.0122
men_df['price(USD)'] = men_df['price(USD)'].round(2)
men_df.head()

In [ ]:
men_df.info()

**Now its time to do the same with the women dataset**

## **1.2) Women dataset**

In [ ]:
women_combined_df_raw.columns

In [ ]:
# creating a copy of the raw dataset and then removing any unwanted columns

women_df = women_combined_df_raw.copy()
women_df = women_df.drop(columns=['Unnamed: 0', 'Link', 'Product_Image'])
women_df.columns = women_df.columns.str.lower()
women_df

In [ ]:
# checking missing data in the price column 

women_df['price'].isna().value_counts()

there is no missing values in the price column

In [ ]:
# checking missing data in the product_name column 
women_df[women_df['product_name'].isna()]

There is only a 3 missing values in the 'product_name' column and 0 missing values in the 'price' column. I'll keep the rows.

Transforming the column 'price'

In [ ]:
# renaming the column price
women_df = women_df.rename(columns={'price' : 'original_price(INR)'})

# removing characters
women_df['original_price(INR)'] = women_df['original_price(INR)'].str.replace('₹ ', '')\
                                                                 .str.replace(',', '')
women_df['original_price(INR)'] = women_df['original_price(INR)'].str.strip()

# converting to float
women_df['original_price(INR)'] = pd.to_numeric(women_df['original_price(INR)'], errors='coerce')

In [ ]:
women_df[women_df['original_price(INR)'].isna()]

The action of converting the column to float resulted in 4 rows in the 'original_price(INR)' column having NaN values, and in these rows, the 'product_name' and 'details' columns also do not contain values, so I will remove both.

In [ ]:
women_df = women_df.dropna(subset=['original_price(INR)'])

# checking missing values again
women_df[women_df['original_price(INR)'].isna()]

Creating the price(USD) column

In [ ]:
women_df['price(USD)'] = women_df.iloc[:, 2] * 0.0122
women_df['price(USD)'] = women_df['price(USD)'].round(2)
women_df.head()

In [ ]:
women_df.info()

# **<div id="edaLink"> 2) Exploratory Analysis </div>**

* [2.1) How many items are there per category?](#2.1)
* [2.2) What is the average price per category?](#2.2)
* [2.3) What is the total price per category?](#2.3)
* [2.4) What is the most expensive item per category?](#2.4)
* [2.5) How many categories are?](#2.5)
* [2.6) How many items are in total?](#2.6)

### **<div id="2.1"> 2.1) How many items are there per category? </div>**

Count of items per category in the Men dataset:

In [ ]:
# creating a subset of the data with 'product_category' count only

men_count_items = men_df['product_category'].value_counts()
men_count_items = men_count_items.sort_values(ascending=True)
men_count_items

In [ ]:
colors = ['BuPu', 'BuPu_r', 'YlGn', 'YlGn_r']

plt.figure(figsize=(10, 7))
men_count_items.plot(kind='barh', cmap=colors[3])
plt.title("Count of items per category in the Men dataset")
plt.xlabel("count")
plt.ylabel("Category")
plt.show()

The same with Women dataset now:

In [ ]:
women_count_items = women_df['product_category'].value_counts()
women_count_items = women_count_items.sort_values(ascending=True)

# the plot of women dataset
plt.figure(figsize=(10, 7))
women_count_items.plot(kind='barh', cmap=colors[1])
plt.title("Count of items per category in the Women dataset")
plt.xlabel("count")
plt.ylabel("Category")
plt.show()

[Back to Header](#edaLink)

### **<div id="2.2"> 2.2) What is the average price per category? </div>**

**Men dataset:**

In [ ]:
# creating a subset of the data with average price per 'product_category'

men_category_mean_price = pd.DataFrame(men_df.groupby("product_category")["price(USD)"].agg("mean"))
men_category_mean_price['price(USD)'] = men_category_mean_price['price(USD)'].round(2)
men_category_mean_price =  men_category_mean_price.sort_values(by='price(USD)', ascending=False)
men_category_mean_price.style.background_gradient(cmap=colors[2], high=0.3)

**Women dataset:**

In [ ]:
women_category_mean_price = pd.DataFrame(women_df.groupby("product_category")["price(USD)"].agg("mean"))
women_category_mean_price['price(USD)'] = women_category_mean_price['price(USD)'].round(2)
women_category_mean_price = women_category_mean_price.sort_values(by='price(USD)', ascending=False)
women_category_mean_price.style.background_gradient(cmap=colors[0], high=0.3)

[Back to Header](#edaLink)

### **<div id="2.3"> 2.3) What is the total price per category? </div>**

**Men dataset:**

In [ ]:
# creating a subset of the data with sum of the price per 'product_category'

men_category_sum_price = pd.DataFrame(men_df.groupby("product_category")["price(USD)"].agg("sum"))
men_category_sum_price.sort_values(by='price(USD)', ascending=False, inplace=True)
men_category_sum_price.style.background_gradient(cmap=colors[2], high=0.3)

The category with the highest total sum of values is 'SHOES' with a total value of: 10,103.64 USD.

**Women dataset:**

In [ ]:
women_category_sum_price = pd.DataFrame(women_df.groupby("product_category")["price(USD)"].agg("sum"))
women_category_sum_price.sort_values(by='price(USD)', ascending=False, inplace=True)
women_category_sum_price.style.background_gradient(cmap=colors[0], high=0.3)

The category with the highest total sum of values is 'DRESSES_JUMPSUITS' with a total value of: 34,639.16 USD.

[Back to Header](#edaLink)

### **<div id="2.4"> 2.4) What is the most expensive item per category? </div>**

Men dataset:

In [ ]:
# creating a dataframe with the most expensive items by category
men_max_price_id = men_df.groupby('product_category')['price(USD)'].idxmax()
expensive_items_men_df = men_df.loc[men_max_price_id].sort_values(by='price(USD)')

# creating a bar plot
plt.figure(figsize=(15, 20),)
plt.barh(expensive_items_men_df['product_category'], expensive_items_men_df['price(USD)'],
         color='darkgreen', height=1)
plt.xlabel('Price(USD)', fontsize=15)
plt.xticks(np.arange(0, 251, 25), fontsize=13)
plt.title('Most Expensive Items by Category in the Men dataset.', fontsize=16)

# Using the product names to create labels for the bars
for index, row in expensive_items_men_df.iterrows():
    plt.text(row['price(USD)'], row['product_category'], row['product_name'],
             ha='left', fontweight='bold')

plt.show()

**Women dataset:**

In [ ]:
# creating a dataframe with the most expensive items by category
women_max_price_id = women_df.groupby('product_category')['price(USD)'].idxmax()
expensive_items_women_df = women_df.loc[women_max_price_id].sort_values(by='price(USD)')

# creating a bar plot
plt.figure(figsize=(15, 20),)
plt.barh(expensive_items_women_df['product_category'], expensive_items_women_df['price(USD)'],
         color='indigo', height=1)
plt.xlabel('Price(USD)', fontsize=15)
plt.xticks(np.arange(0, 251, 25), fontsize=13)
plt.title('Most Expensive Items by Category in the Women dataset.', fontsize=16)

# Using the product names to create labels for the bars
for index, row in expensive_items_women_df.iterrows():
    plt.text(row['price(USD)'], row['product_category'], row['product_name'],
             ha='left', fontweight='bold')

plt.show()

[Back to Header](#edaLink)

### **<div id="2.5"> 2.5) How many categories are? </div>**

In [ ]:
print("Unique categories in the Men dataset: ", men_df['product_category'].nunique())

In [ ]:
print("Unique categories in the Women dataset: ", women_df['product_category'].nunique())

[Back to Header](#edaLink)

### **<div id="2.6"> 2.6) How many items are in total? </div>**

In [ ]:
# creating a dataframe with the total product count
total_product_count = {'labels': ['Men', 'Women'],
                      'product_count': [men_df['product_name'].count(),
                                        women_df['product_name'].count()]}
total_product_count = pd.DataFrame(total_product_count)
total_product_count

In [ ]:
# creating a pie chart
explode = [0, 0.1]
_, _, autotexts = plt.pie(total_product_count.product_count, labels=total_product_count.labels,
                          explode=explode, colors=['darkgreen', 'indigo'], shadow=True, startangle=90,
                          autopct='%1.1f%%', textprops={'color': 'black', 'weight': 'bold'})

# adding the internal labels
for autotext in autotexts:
    autotext.set_color('white') # choosing the internal label color

plt.title("Total Product Count")

plt.axis('equal')
plt.show()

This was a basic analysis; it can still be more accurate because i didn't remove outliers and duplicate values.

Thx for your time and pls upvote the notebook

[Back to Header](#edaLink)